In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import csv
import random
from datetime import datetime
from reportlab.lib import colors
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.styles import getSampleStyleSheet
import os

class ExamSchedulerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Générateur de Planning de Surveillance")
        self.root.attributes('-fullscreen', True)
        self.root.configure(bg='#f0f4f8')

        # Données
        self.template_rows = []
        self.filled_rows = []
        self.teachers = []
        self.slots = []
        self.teacher_hours = {}

        # Styles
        self.style = ttk.Style()
        self.style.configure("TButton", font=("Arial", 12), padding=10)
        self.style.configure("TLabel", font=("Arial", 12), background='#f0f4f8')
        self.style.map("TButton", background=[('active', '#4CAF50')])

        # Interface
        self.create_widgets()

    def create_widgets(self):
        # Cadre principal
        main_frame = ttk.Frame(self.root, padding=20)
        main_frame.grid(row=0, column=0, sticky="nsew")
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)

        # Section Importation CSV
        import_frame = ttk.LabelFrame(main_frame, text="Importation des Données", padding=10)
        import_frame.grid(row=0, column=0, sticky="ew", padx=5, pady=5)
        ttk.Button(import_frame, text="Planning", command=self.import_input_csv).grid(row=0, column=0, padx=5)
        ttk.Button(import_frame, text="Surveillants", command=self.import_surveillants_csv).grid(row=0, column=1, padx=5)
        ttk.Button(import_frame, text="Générer Planning", command=self.generate_schedule).grid(row=0, column=2, padx=5)

        # Section Données Importées
        data_frame = ttk.LabelFrame(main_frame, text="Données Importées", padding=10)
        data_frame.grid(row=1, column=0, sticky="nsew", padx=5, pady=5)
        self.data_tree = ttk.Treeview(data_frame, columns=("Date", "Heure", "Local", "Modules", "Responsables", "Surveillants Nécessaires"), show="headings")
        self.data_tree.heading("Date", text="Date")
        self.data_tree.heading("Heure", text="Heure")
        self.data_tree.heading("Local", text="Local")
        self.data_tree.heading("Modules", text="Modules")
        self.data_tree.heading("Responsables", text="Responsables")
        self.data_tree.heading("Surveillants Nécessaires", text="Surveillants Nécessaires")
        self.data_tree.grid(row=0, column=0, sticky="nsew")
        data_frame.columnconfigure(0, weight=1)
        data_frame.rowconfigure(0, weight=1)

        # Section Planning Généré
        schedule_frame = ttk.LabelFrame(main_frame, text="Planning Généré", padding=10)
        schedule_frame.grid(row=1, column=1, sticky="nsew", padx=5, pady=5)
        self.schedule_tree = ttk.Treeview(schedule_frame, columns=("Date", "Heure", "Local", "Modules", "Responsables", "Surveillants Assignés"), show="headings")
        self.schedule_tree.heading("Date", text="Date")
        self.schedule_tree.heading("Heure", text="Heure")
        self.schedule_tree.heading("Local", text="Local")
        self.schedule_tree.heading("Modules", text="Modules")
        self.schedule_tree.heading("Responsables", text="Responsables")
        self.schedule_tree.heading("Surveillants Assignés", text="Surveillants Assignés")
        self.schedule_tree.tag_configure('amphi', background='lightgreen')
        self.schedule_tree.tag_configure('salle', background='lightyellow')
        self.schedule_tree.grid(row=0, column=0, sticky="nsew")
        schedule_frame.columnconfigure(0, weight=1)
        schedule_frame.rowconfigure(0, weight=1)

        # Section Statistiques et Exportation
        stats_export_frame = ttk.LabelFrame(main_frame, text="Statistiques et Exportation", padding=10)
        stats_export_frame.grid(row=2, column=0, columnspan=2, sticky="ew", padx=5, pady=5)
        ttk.Button(stats_export_frame, text="Afficher Heures par Prof", command=self.show_teacher_hours).grid(row=0, column=0, padx=5)
        ttk.Button(stats_export_frame, text="Générer Convocations PDF", command=self.generate_convocation_pdfs).grid(row=0, column=1, padx=5)
        ttk.Button(stats_export_frame, text="Exporter CSV", command=self.export_csv).grid(row=0, column=2, padx=5)
        ttk.Button(stats_export_frame, text="Exporter PDF", command=self.export_pdf).grid(row=0, column=3, padx=5)
        ttk.Button(stats_export_frame, text="Quitter", command=self.root.quit).grid(row=0, column=4, padx=5)

        # Configuration responsive
        main_frame.columnconfigure(0, weight=1)
        main_frame.columnconfigure(1, weight=1)
        main_frame.rowconfigure(1, weight=1)

    def import_surveillants_csv(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            try:
                with open(file_path, newline='', encoding='utf-8') as csvfile:
                    reader = csv.DictReader(csvfile)
                    self.teachers = [row["NOM ET PRENOM"].strip() for row in reader if row["NOM ET PRENOM"].strip()]
                messagebox.showinfo("Succès", "Liste des surveillants importée avec succès !")
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur lors de l'importation des surveillants : {e}")

    def import_input_csv(self):
        file_path = filedialog.askopenfilename(filetypes=[("CSV files", "*.csv")])
        if file_path:
            try:
                with open(file_path, newline='', encoding='utf-8') as csvfile:
                    reader = list(csv.DictReader(csvfile))
                    self.template_rows = reader
                    self.parse_slots()
                    self.update_data_tree()
                messagebox.showinfo("Succès", "Données d'entrée importées avec succès !")
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur lors de l'importation : {e}")

    def parse_slots(self):
        self.slots = []
        current_key = None
        surveillant_count = 0
        responsibles = []
        modules = []

        for row in self.template_rows:
            date = row['DATE']
            heure = row['HEURE']
            local = row['LOCAL']
            module = row['FILIERE - SEMESTRE - MODULE']
            nom = row['NOM ET PRENOM']
            qualite = row['QUALITE']
            statut = row['Statut']

            key = (date, heure, local)

            if key != current_key:
                if current_key:
                    self.slots.append({
                        'Date': current_key[0],
                        'Heure': current_key[1],
                        'Local': current_key[2],
                        'Modules': ', '.join(set(m for m, _, _ in responsibles if m != '-')),
                        'Responsibles': ', '.join(set(n for _, n, _ in responsibles if n)),
                        'NumSurveillants': surveillant_count,
                        'Type': 'amphi' if 'AMPHI' in current_key[2].upper() else 'salle',
                        'Forbidden': set(n for _, n, _ in responsibles if n)
                    })
                current_key = key
                surveillant_count = 0
                responsibles = []

            if qualite == 'SURVEILLANT' and not nom:
                surveillant_count += 1
            elif 'RESPONSABLE' in qualite and nom:
                if module != '-':
                    is_et = 'ET SURVEILLANT' in qualite
                    responsibles.append((module, nom, statut))

        if current_key:
            self.slots.append({
                'Date': current_key[0],
                'Heure': current_key[1],
                'Local': current_key[2],
                'Modules': ', '.join(set(m for m, _, _ in responsibles if m != '-')),
                'Responsibles': ', '.join(set(n for _, n, _ in responsibles if n)),
                'NumSurveillants': surveillant_count,
                'Type': 'amphi' if 'AMPHI' in current_key[2].upper() else 'salle',
                'Forbidden': set(n for _, n, _ in responsibles if n)
            })

    def update_data_tree(self):
        for item in self.data_tree.get_children():
            self.data_tree.delete(item)
        for slot in self.slots:
            self.data_tree.insert("", "end", values=(
                slot["Date"], slot["Heure"], slot["Local"], slot["Modules"], slot["Responsibles"], slot["NumSurveillants"]
            ))

    def generate_schedule(self):
        if not self.teachers:
            messagebox.showerror("Erreur", "Veuillez importer la liste des surveillants d'abord.")
            return
        if not self.template_rows:
            messagebox.showerror("Erreur", "Veuillez importer les données d'entrée d'abord.")
            return
        try:
            self.filled_rows = [row.copy() for row in self.template_rows]
            self.teacher_hours = {t: 0 for t in self.teachers}
            time_keys = set((row['DATE'], row['HEURE']) for row in self.filled_rows)

            empty_positions = [(i, row) for i, row in enumerate(self.filled_rows) if row['QUALITE'] == 'SURVEILLANT' and not row['NOM ET PRENOM']]

            for date, heure in time_keys:
                used_teachers = set(row['NOM ET PRENOM'] for row in self.filled_rows if row['DATE'] == date and row['HEURE'] == heure and row['NOM ET PRENOM'])
                avail = list(set(self.teachers) - used_teachers)
                empties_this_time = [(i, row) for i, row in empty_positions if row['DATE'] == date and row['HEURE'] == heure]
                random.shuffle(empties_this_time)

                local_to_forbidden = {}
                for slot in self.slots:
                    if slot['Date'] == date and slot['Heure'] == heure:
                        local_to_forbidden[slot['Local']] = slot.get('Forbidden', set())

                for i, row in empties_this_time:
                    local = row['LOCAL']
                    forbidden = local_to_forbidden.get(local, set())
                    candidates = [t for t in avail if t not in forbidden]
                    if not candidates:
                        messagebox.showwarning("Avertissement", f"Pas assez de surveillants disponibles pour {date} {heure} {local}.")
                        return
                    candidates.sort(key=lambda t: self.teacher_hours[t])
                    t = candidates[0]
                    self.filled_rows[i]['NOM ET PRENOM'] = t
                    self.teacher_hours[t] += 2  # Chaque créneau = 2 heures
                    avail.remove(t)

            self.update_schedule_tree()
            messagebox.showinfo("Succès", "Planning généré avec succès !")
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la génération : {str(e)}")

    def update_schedule_tree(self):
        for item in self.schedule_tree.get_children():
            self.schedule_tree.delete(item)
        assigned_slots = []
        for slot in self.slots:
            date = slot['Date']
            heure = slot['Heure']
            local = slot['Local']
            surveillants = [row['NOM ET PRENOM'] for row in self.filled_rows if row['DATE'] == date and row['HEURE'] == heure and row['LOCAL'] == local and row['QUALITE'] == 'SURVEILLANT' and row['NOM ET PRENOM']]
            assigned = ', '.join(surveillants) if surveillants else "Aucun"
            tag = slot['Type']
            assigned_slots.append({
                'Date': date,
                'Heure': heure,
                'Local': local,
                'Modules': slot['Modules'],
                'Responsables': slot['Responsibles'],
                'Surveillants': assigned
            })
            self.schedule_tree.insert("", "end", values=(
                date, heure, local, slot['Modules'], slot['Responsibles'], assigned
            ), tags=(tag,))

    def show_teacher_hours(self):
        hours_window = tk.Toplevel(self.root)
        hours_window.title("Heures par Professeur")
        hours_window.geometry("400x300")

        tree = ttk.Treeview(hours_window, columns=("Professeur", "Heures"), show="headings")
        tree.heading("Professeur", text="Professeur")
        tree.heading("Heures", text="Heures")
        tree.grid(row=0, column=0, sticky="nsew")

        for teacher, hours in self.teacher_hours.items():
            tree.insert("", "end", values=(teacher, hours))

        hours_window.columnconfigure(0, weight=1)
        hours_window.rowconfigure(0, weight=1)

    def generate_convocation_pdfs(self):
        if not self.filled_rows:
            messagebox.showerror("Erreur", "Générez le planning d'abord.")
            return
        try:
            folder_path = filedialog.askdirectory(title="Sélectionner le dossier de destination")
            if not folder_path:
                return

            for teacher in set(row['NOM ET PRENOM'] for row in self.filled_rows if row['NOM ET PRENOM']):
                assignments = [row for row in self.filled_rows if row['NOM ET PRENOM'] == teacher]
                if not assignments:
                    continue
                file_path = os.path.join(folder_path, f"convocation_{teacher}.pdf")
                doc = SimpleDocTemplate(file_path, pagesize=A4)
                styles = getSampleStyleSheet()
                story = []

                title = Paragraph(f"Convocation de Surveillance - {teacher}", styles['Heading1'])
                story.append(title)
                story.append(Paragraph(f"Date : {datetime.now().strftime('%Y-%m-%d %H:%M')}", styles['Normal']))
                story.append(Paragraph("Liste des surveillances assignées :", styles['Heading2']))

                data = [["Date", "Heure", "Local", "Module"]]
                for assign in assignments:
                    data.append([assign['DATE'], assign['HEURE'], assign['LOCAL'], assign['FILIERE - SEMESTRE - MODULE']])
                table = Table(data)
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 12),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ]))
                story.append(table)

                doc.build(story)

            messagebox.showinfo("Succès", f"Convocations générées avec succès dans {folder_path} !")
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la génération des convocations : {e}")

    def export_csv(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
        if file_path:
            try:
                with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=self.filled_rows[0].keys())
                    writer.writeheader()
                    writer.writerows(self.filled_rows)
                messagebox.showinfo("Succès", "Planning exporté en CSV avec succès !")
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur lors de l'exportation CSV : {e}")

    def export_pdf(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
        if file_path:
            try:
                doc = SimpleDocTemplate(file_path, pagesize=A4)
                data = [["Date", "Heure", "Local", "Module", "Nom", "Qualite", "Statut"]]
                for row in self.filled_rows:
                    data.append([row["DATE"], row["HEURE"], row["LOCAL"], row["FILIERE - SEMESTRE - MODULE"], row["NOM ET PRENOM"], row["QUALITE"], row["Statut"]])
                table = Table(data)
                table.setStyle(TableStyle([
                    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('FONTSIZE', (0, 0), (-1, 0), 12),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black),
                ]))
                doc.build([table])
                messagebox.showinfo("Succès", "Planning exporté en PDF avec succès !")
            except Exception as e:
                messagebox.showerror("Erreur", f"Erreur lors de l'exportation PDF : {e}")

if __name__ == "__main__":
    root = tk.Tk()
    app = ExamSchedulerApp(root)
    root.mainloop()